In [64]:
from IPython.utils import io
import os
import subprocess
import tqdm.notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import torch
from collections import Counter

# BoTorch / GPyTorch 関連
from botorch import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf
from mpl_toolkits.mplot3d import Axes3D  # 必要に応じて

# Plot周りの設定
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'
plt.rcParams["figure.dpi"] = 100  # 高解像度設定

In [65]:
def styblinski_tang(x, noise_std=0.00001):
    """
    x: shape (..., d) のテンソル（d>=5を仮定）
       目的関数の計算には最初の5次元のみを利用する。
    noise_std: ノイズの標準偏差
    """
    if torch.is_tensor(x):
        x_val = x[..., :5]  # shape (..., 5)
    else:
        x_val = torch.tensor(x[..., :5], dtype=torch.float32)
    result = 0.5 * torch.sum(x_val ** 4 - 16 * x_val ** 2 + 5 * x_val, dim=-1)
    noise = torch.randn_like(result) * noise_std
    return result + noise


# 提案手法: ECI_BO_Bandit

In [66]:
class ECI_BO_Bandit:
    def __init__(self, X, objective_function, bounds, n_initial, n_max, dim, coordinate_ratio=0.5):
        """
        X: 初期サンプル点 (n_initial, dim)
        objective_function: 目的関数 (styblinski_tang など)
        bounds: 探索の下限・上限 (2, dim) のテンソル
        n_initial: 初期サンプル数
        n_max: 反復最大回数
        dim: 入力次元 (例: 50や100など)
        coordinate_ratio: 座標方向ベクトルを何割使うか
        """
        dtype = torch.float
        self.num_arms = dim  # 各イテレーションで生成される候補方向の数

        # 線形バンディット用のパラメータ A, b の初期化
        self.A = torch.eye(dim, dtype=dtype)
        self.b = torch.zeros(dim, dtype=dtype)

        self.objective_function = objective_function
        self.bounds = bounds.to(dtype=dtype)
        self.n_initial = n_initial
        self.n_max = n_max
        self.dim = dim
        self.X = X.to(dtype=dtype)
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None

        self.eval_history = []          # 各反復でのbest_valueの履歴
        self.arm_selection_history = [] # 選択された arm のインデックスの履歴
        self.optimization_history = []  # 探索した新しい点の履歴
        self.saved_plot_files = []      # GP後方分布のプロット保存用

        self.coordinate_ratio = coordinate_ratio

        # UCBおよび線形バンディットの解析用パラメータ（文献に基づく一例）
        self.sigma = 0.1
        self.L = 5.0
        self.lambda_reg = 1.0
        self.delta = 0.1
        self.S = 10.0

        self.selected_direction_history = []  # 選択された方向ベクトル
        self.theta_history = []               # 推定パラメータ theta
        self.reward_history = []              # 各イテレーションの報酬
        self.iteration_table_data = []

        # 選択された方向の絶対値を累積（後で棒グラフなどに使う）
        self.direction_sum_abs = torch.zeros(dim, dtype=dtype)

    def update_model(self):
        """
        BoTorchのSingleTaskGPを更新する(全アルゴリズムで共通)。
        """
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1]),
            noise_constraint=1e-2
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)

    def initialize(self):
        """
        初期点 X に対して目的関数を評価し、GPモデルを初期化する。
        """
        y_val = self.objective_function(self.X)
        if y_val.dtype != torch.float:
            y_val = y_val.float()
        self.Y = y_val.unsqueeze(-1)

        # GPモデル更新
        self.update_model()

        # 事後平均から最良点を選ぶ
        posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
        best_index = posterior_mean.argmin()
        self.best_value = posterior_mean[best_index].item()
        self.best_point = self.X[best_index]

        self.eval_history = [self.best_value] * self.n_initial

    def generate_arms(self):
        """
        各イテレーションで self.num_arms 本の候補方向ベクトルを生成。
        うち (coordinate_ratio * dim) 本は座標方向ベクトル、残りはランダム方向。
        """
        dtype = torch.float
        num_coord = int(math.floor(self.coordinate_ratio * self.dim))
        # 上限をdim以内に収める
        if num_coord > self.dim:
            num_coord = self.dim

        # (1) 座標方向ベクトル
        random_indices = np.random.choice(self.dim, num_coord, replace=False)
        coordinate_arms = []
        for idx in random_indices:
            e_i = torch.zeros(self.dim, dtype=dtype)
            e_i[idx] = 1.0
            coordinate_arms.append(e_i)
        if len(coordinate_arms) > 0:
            coordinate_arms = torch.stack(coordinate_arms, dim=0)
        else:
            coordinate_arms = torch.zeros(0, self.dim, dtype=dtype)

        # (2) ランダム方向ベクトル
        num_random = self.dim - num_coord
        if num_random > 0:
            random_arms = torch.randn(num_random, self.dim, dtype=dtype)
            random_arms = random_arms / random_arms.norm(dim=1, keepdim=True)
        else:
            random_arms = torch.zeros(0, self.dim, dtype=dtype)

        # (3) 結合
        arms = torch.cat([coordinate_arms, random_arms], dim=0)

        return arms

    def select_arm(self, total_iterations):
        """
        線形バンディット(LinUCB)により、候補方向の中から最良と思われるものを選択する。
        """
        A_inv = torch.inverse(self.A)
        theta = A_inv @ self.b
        self.theta_history.append(theta.clone())

        p_values = []
        beta_t = (
            self.sigma * math.sqrt(self.dim * math.log((1 + total_iterations * (self.L ** 2) / self.lambda_reg) / self.delta))
            + math.sqrt(self.lambda_reg) * self.S
        )

        for i in range(self.num_arms):
            x_arm = self.arms_features[i].view(-1, 1)
            mean = (theta.view(1, -1) @ x_arm).item()
            var = (x_arm.t() @ A_inv @ x_arm).item()
            ucb = mean + beta_t * math.sqrt(var)
            p_values.append(ucb)

        return int(np.argmax(p_values))

    def propose_new_x(self, direction):
        """
        選択された方向 direction に沿って 1次元最適化し、新たな点 new_x を提案する。
        """
        dtype = torch.float
        ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)

        def eci_func(x):
            x_scalar = x.squeeze(-1).squeeze(-1)
            full_x = self.best_point.clone().unsqueeze(0) + x_scalar[:, None] * direction
            full_x = full_x.unsqueeze(1)
            return ei(full_x)

        # direction に沿った t の範囲を bounds でクリップ
        lower_list = []
        upper_list = []
        for i in range(self.dim):
            d_i = direction[i].item()
            if abs(d_i) < 1e-8:
                continue
            if d_i > 0:
                lower_list.append((self.bounds[0][i] - self.best_point[i]).item() / d_i)
                upper_list.append((self.bounds[1][i] - self.best_point[i]).item() / d_i)
            else:
                lower_list.append((self.bounds[1][i] - self.best_point[i]).item() / d_i)
                upper_list.append((self.bounds[0][i] - self.best_point[i]).item() / d_i)
        if lower_list:
            candidate_lower_bound = max(lower_list)
            candidate_upper_bound = min(upper_list)
            # 下限が上限を超える場合は t=0 に固定
            if candidate_lower_bound > candidate_upper_bound:
                candidate_lower_bound = candidate_upper_bound = 0.0
        else:
            candidate_lower_bound = -5.0
            candidate_upper_bound = 5.0

        one_d_bounds = torch.tensor([[candidate_lower_bound], [candidate_upper_bound]], dtype=dtype)

        candidate, acq_value = optimize_acqf(
            eci_func,
            one_d_bounds,
            q=1,
            num_restarts=10,
            raw_samples=100,
        )

        new_x = self.best_point.clone() + candidate.squeeze() * direction
        return new_x, candidate, acq_value

    def optimize(self):
        """
        メインの最適化ループ。GPによる局所探索 + 線形バンディットによる方向選択。
        """
        self.initialize()
        self.optimization_history = []
        n = self.n_initial
        total_iterations = 1
        dtype = torch.float

        while n < self.n_max:
            # 候補方向の生成
            self.arms_features = self.generate_arms()

            # 線形バンディットで方向を選択
            selected_arm = self.select_arm(total_iterations)
            self.arm_selection_history.append(selected_arm)
            direction = self.arms_features[selected_arm]
            self.selected_direction_history.append(direction.clone())

            # ベイズ最適化 (EI) による 1次元最適化で新点を提案
            new_x, candidate, acq_value = self.propose_new_x(direction)
            new_y = self.objective_function(new_x.unsqueeze(0)).unsqueeze(-1).to(dtype=dtype)

            # データを更新
            self.X = torch.cat([self.X, new_x.unsqueeze(0)])
            self.Y = torch.cat([self.Y, new_y])
            self.optimization_history.append(new_x.clone())

            # 線形バンディットの情報更新
            x_arm = direction.view(-1, 1)
            self.A = self.A + x_arm @ x_arm.t()

            # 報酬を exp(-|p - y|) で定義 (pはGP事後平均)
            p = self.model.posterior(new_x.unsqueeze(0)).mean.squeeze()
            reward = 1 + - torch.exp(-torch.abs(p - new_y.squeeze())).item()
            self.reward_history.append(reward)
            self.b = self.b + reward * direction

            # GPモデルを更新し、best_point を再計算
            self.update_model()
            posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            best_index = posterior_mean.argmin()
            self.best_value = posterior_mean[best_index].item()
            self.best_point = self.X[best_index]

            self.eval_history.append(self.best_value)
            self.direction_sum_abs += direction.abs()

            # カーネルハイパラなどの記録 (必要に応じて)
            kernel = self.model.covar_module
            lengthscale = kernel.base_kernel.lengthscale.detach().cpu().numpy()
            outputscale = kernel.outputscale.detach().cpu().numpy()

            iteration_data = {
                "iteration": total_iterations,
                "x": new_x.detach().cpu().numpy(),
                "y": new_y.item(),
                "best_value": self.best_value,
                "candidate_parameta": candidate.squeeze(),
                "kernel_lengthscale": lengthscale,
                "kernel_outputscale": outputscale,
                "reward": reward,
            }
            self.iteration_table_data.append(iteration_data)

            n += 1
            total_iterations += 1

        return self.best_point, self.best_value

# シンプルなDropout戦略による BO

In [67]:
class Dropout_BO:
    def __init__(self, X, objective_function, bounds, n_initial, n_max, dim, dropout_rate=0.5):
        dtype = torch.float
        self.objective_function = objective_function
        self.full_dim = dim
        self.dropout_rate = dropout_rate
        self.bounds = bounds.to(dtype=dtype)
        self.n_initial = n_initial
        self.n_max = n_max
        self.X = X.to(dtype=dtype)
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []

    def update_model(self):
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1]),
            noise_constraint=1e-1
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)

    def initialize(self):
        y_val = self.objective_function(self.X)
        if y_val.dtype != torch.float:
            y_val = y_val.float()
        self.Y = y_val.unsqueeze(-1)
        self.update_model()

        posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
        best_index = posterior_mean.argmin()
        self.best_value = posterior_mean[best_index].item()
        self.best_point = self.X[best_index]
        self.eval_history = [self.best_value] * self.n_initial

    def optimize(self):
        self.initialize()
        n = self.n_initial
        dtype = torch.float

        while n < self.n_max:
            num_vars = int((1 - self.dropout_rate) * self.full_dim)
            selected_dims_np = np.sort(np.random.choice(self.full_dim, num_vars, replace=False))
            selected_dims = torch.tensor(selected_dims_np, dtype=torch.long, device=self.X.device)

            def dropout_ei(x):
                x_scalar = x.squeeze(-1).squeeze(-1)
                best_point = self.best_point.clone().detach()
                full_x = best_point.unsqueeze(0).expand(x_scalar.shape[0], -1)
                full_x[:, selected_dims] = full_x[:, selected_dims] + x_scalar.unsqueeze(1)
                full_x = full_x.unsqueeze(1)
                ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
                return ei(full_x)

            one_d_bounds = torch.tensor([[-5.0], [5.0]], dtype=dtype)
            candidate, acq_value = optimize_acqf(
                dropout_ei,
                one_d_bounds,
                q=1,
                num_restarts=10,
                raw_samples=100,
            )
            new_x = self.best_point.clone()
            new_x[selected_dims] = new_x[selected_dims] + candidate.squeeze()
            new_x = torch.clamp(new_x, self.bounds[0], self.bounds[1])

            new_y = self.objective_function(new_x.unsqueeze(0)).unsqueeze(-1).to(dtype=dtype)
            self.X = torch.cat([self.X, new_x.unsqueeze(0)])
            self.Y = torch.cat([self.Y, new_y])

            self.update_model()
            posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            best_index = posterior_mean.argmin()
            self.best_value = posterior_mean[best_index].item()
            self.best_point = self.X[best_index]
            self.eval_history.append(self.best_value)

            n += 1

        return self.best_point, self.best_value


# ECI_BO (座標ごとに EI を1次元最適化する例)

In [68]:
class ECI_BO:
    def __init__(self, X, objective_function, bounds, n_initial, n_max, dim):
        dtype = torch.float
        self.objective_function = objective_function
        self.dim = dim
        self.bounds = bounds.to(dtype=dtype)
        self.n_initial = n_initial
        self.n_max = n_max
        self.X = X.to(dtype=dtype)
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []

    def update_model(self):
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1]),
            noise_constraint=1e-2
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)

    def initialize(self):
        y_val = self.objective_function(self.X)
        if y_val.dtype != torch.float:
            y_val = y_val.float()
        self.Y = y_val.unsqueeze(-1)

        self.update_model()
        posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
        best_index = posterior_mean.argmin()
        self.best_value = posterior_mean[best_index].item()
        self.best_point = self.X[best_index].view(-1)
        self.eval_history = [self.best_value] * self.n_initial

    def optimize(self):
        self.initialize()
        n = self.n_initial
        dtype = torch.float

        while n < self.n_max:
            best_improv = float('inf')
            best_candidate = 0.0
            best_axis = 0

            for axis in range(self.dim):
                def axis_ei(x):
                    x_scalar = x.view(-1)  # shape: [batch_size]
                    bp = self.best_point.view(-1)  # shape: [dim]
                    batch_size = int(x_scalar.shape[0])
                    # batch_size 個の候補点を作成
                    full_x = bp.unsqueeze(0).repeat(batch_size, 1)
                    full_x[:, axis] = full_x[:, axis] + x_scalar
                    full_x = full_x.unsqueeze(1)  # shape: [batch_size, 1, dim]
                    ei_val = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
                    return ei_val(full_x)

                one_d_bounds = torch.tensor([[-5.0], [5.0]], dtype=dtype)
                candidate, acq_value = optimize_acqf(
                    axis_ei,
                    one_d_bounds,
                    q=1,
                    num_restarts=5,
                    raw_samples=50,
                )
                # axis_ei(candidate) は shape=[1] のテンソルになるので .item() でスカラー取得
                val = axis_ei(candidate).item()

                if val < best_improv:
                    best_improv = val
                    # candidate も同様に item() でスカラーに変換
                    best_candidate = candidate.item()
                    best_axis = axis

            new_x = self.best_point.clone()
            # += の右辺をスカラーにする（ベクトルだと shape が合わずエラー）
            new_x[best_axis] += best_candidate
            new_x = torch.clamp(new_x, self.bounds[0], self.bounds[1])

            new_y = self.objective_function(new_x.unsqueeze(0)).unsqueeze(-1).to(dtype=dtype)
            self.X = torch.cat([self.X, new_x.unsqueeze(0)])
            self.Y = torch.cat([self.Y, new_y])

            self.update_model()
            posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            best_index = posterior_mean.argmin()
            self.best_value = posterior_mean[best_index].item()
            self.best_point = self.X[best_index].view(-1)
            self.eval_history.append(self.best_value)

            n += 1

        return self.best_point, self.best_value

# SAASBO (簡易実装版: 実際のSAAS priorではなくARDカーネルで代用)

In [69]:
class SAASBO_BO:
    def __init__(self, X, objective_function, bounds, n_initial, n_max, dim):
        dtype = torch.float
        self.objective_function = objective_function
        self.dim = dim
        self.bounds = bounds.to(dtype=dtype)
        self.n_initial = n_initial
        self.n_max = n_max
        self.X = X.to(dtype=dtype)
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []

    def update_model(self):
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1]),
            noise_constraint=1e-2
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)

    def initialize(self):
        y_val = self.objective_function(self.X)
        if y_val.dtype != torch.float:
            y_val = y_val.float()
        self.Y = y_val.unsqueeze(-1)
        self.update_model()

        posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
        best_index = posterior_mean.argmin()
        self.best_value = posterior_mean[best_index].item()
        self.best_point = self.X[best_index]
        self.eval_history = [self.best_value] * self.n_initial

    def optimize(self):
        self.initialize()
        n = self.n_initial
        dtype = torch.float

        while n < self.n_max:
            ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
            # 全次元にわたる空間(bounds)で最適化
            candidate, acq_value = optimize_acqf(
                ei,
                bounds=self.bounds,
                q=1,
                num_restarts=10,
                raw_samples=100,
            )
            # ここではシンプルに "best_point + candidate" にしているが
            # 実際は candidate 自体が [q, d] の最適点である
            new_x = self.best_point.clone() + candidate.squeeze()
            new_x = torch.clamp(new_x, self.bounds[0], self.bounds[1])

            new_y = self.objective_function(new_x.unsqueeze(0)).unsqueeze(-1).to(dtype=dtype)
            self.X = torch.cat([self.X, new_x.unsqueeze(0)])
            self.Y = torch.cat([self.Y, new_y])

            self.update_model()
            posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            best_index = posterior_mean.argmin()
            self.best_value = posterior_mean[best_index].item()
            self.best_point = self.X[best_index].view(-1)
            self.eval_history.append(self.best_value)
            n += 1

        return self.best_point, self.best_value


# ★追加★ 単純な(フル次元)ベイズ最適化
   - ARD RBF + EI + 全次元を一括で最適化

In [70]:
class VanillaBO:
    def __init__(self, X, objective_function, bounds, n_initial, n_max, dim):
        dtype = torch.float
        self.objective_function = objective_function
        self.dim = dim
        self.bounds = bounds.to(dtype=dtype)
        self.n_initial = n_initial
        self.n_max = n_max
        self.X = X.to(dtype=dtype)
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []

    def update_model(self):
        # 全アルゴリズムで共通のGP更新
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1]),
            noise_constraint=1e-2
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)

    def initialize(self):
        y_val = self.objective_function(self.X)
        if y_val.dtype != torch.float:
            y_val = y_val.float()
        self.Y = y_val.unsqueeze(-1)
        self.update_model()

        # 初期点の中で事後平均が最も低いものをbest_pointとする
        posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
        best_index = posterior_mean.argmin()
        self.best_value = posterior_mean[best_index].item()
        self.best_point = self.X[best_index]
        self.eval_history = [self.best_value] * self.n_initial

    def optimize(self):
        self.initialize()
        n = self.n_initial
        dtype = torch.float

        while n < self.n_max:
            # EIを定義
            ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
            # フル次元の bounds で直接最適化
            candidate, acq_value = optimize_acqf(
                ei,
                bounds=self.bounds,
                q=1,
                num_restarts=10,
                raw_samples=100,
            )
            new_x = candidate.squeeze().clone()  # 今回は直接 candidate を採用
            new_x = torch.clamp(new_x, self.bounds[0], self.bounds[1])

            new_y = self.objective_function(new_x.unsqueeze(0)).unsqueeze(-1).to(dtype=dtype)
            self.X = torch.cat([self.X, new_x.unsqueeze(0)])
            self.Y = torch.cat([self.Y, new_y])

            self.update_model()
            posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            best_index = posterior_mean.argmin()
            self.best_value = posterior_mean[best_index].item()
            self.best_point = self.X[best_index].view(-1)
            self.eval_history.append(self.best_value)

            n += 1

        return self.best_point, self.best_value


In [71]:
def generate_initial_points(n_initial, dim, bounds):
    return torch.rand(n_initial, dim) * (bounds[1] - bounds[0]) + bounds[0]

In [ ]:
if __name__ == "__main__":
    # パラメータ設定
    dim = 50
    bounds = torch.tensor([[-5.0] * dim, [5.0] * dim])
    n_initial = 20
    n_iter = 500  # 計算時間などに応じて適宜変更
    n_runs = 50

    # REMBOは削除したのでリストから除外
    # 代わりに "VanillaBO" を追加
    results = {
        "ECI_BO_Bandit": [],
        "VanillaBO": [],
        "Dropout_BO": [],
        "ECI_BO": [],
        "SAASBO_BO": []
    }

    try:
        with tqdm.notebook.tqdm(total=n_runs, bar_format=TQDM_BAR_FORMAT) as pbar:
            with io.capture_output() as captured:
                for run in range(n_runs):
                    X_init = generate_initial_points(n_initial, dim, bounds)

                    algo1 = ECI_BO_Bandit(
                        X_init.clone(),
                        objective_function=styblinski_tang,
                        bounds=bounds,
                        n_initial=n_initial,
                        n_max=n_iter,
                        dim=dim,
                        coordinate_ratio=0.8
                    )
                    algo2 = VanillaBO(
                        X_init.clone(),
                        objective_function=styblinski_tang,
                        bounds=bounds,
                        n_initial=n_initial,
                        n_max=n_iter,
                        dim=dim
                    )
                    algo3 = Dropout_BO(
                        X_init.clone(),
                        objective_function=styblinski_tang,
                        bounds=bounds,
                        n_initial=n_initial,
                        n_max=n_iter,
                        dim=dim,
                        dropout_rate=0.5
                    )
                    algo4 = ECI_BO(
                        X_init.clone(),
                        objective_function=styblinski_tang,
                        bounds=bounds,
                        n_initial=n_initial,
                        n_max=n_iter,
                        dim=dim
                    )
                    algo5 = SAASBO_BO(
                        X_init.clone(),
                        objective_function=styblinski_tang,
                        bounds=bounds,
                        n_initial=n_initial,
                        n_max=n_iter,
                        dim=dim
                    )

                    best_x1, best_f1 = algo1.optimize()
                    best_x2, best_f2 = algo2.optimize()
                    best_x3, best_f3 = algo3.optimize()
                    best_x4, best_f4 = algo4.optimize()
                    best_x5, best_f5 = algo5.optimize()

                    results["ECI_BO_Bandit"].append(algo1.eval_history)
                    results["VanillaBO"].append(algo2.eval_history)
                    results["Dropout_BO"].append(algo3.eval_history)
                    results["ECI_BO"].append(algo4.eval_history)
                    results["SAASBO_BO"].append(algo5.eval_history)

                    pbar.update(1)
    except subprocess.CalledProcessError:
        print(captured)
        raise

    # 可視化
    plt.figure(figsize=(10,6))
    for key, val in results.items():
        # 各試行の評価履歴の最小長さを求め、切り詰めて平均＆標準偏差をとる
        min_length = min(len(hist) for hist in val)
        data = np.array([hist[:min_length] for hist in val])
        mean_history = data.mean(axis=0)
        std_history = data.std(axis=0)
        x_axis = np.arange(min_length)

        # 提案手法(ECI_BO_Bandit)だけ線を太く・色を変えて目立たせる
        if key == "ECI_BO_Bandit":
            plt.plot(x_axis, mean_history, label=key, linewidth=3, color='red')
            plt.fill_between(x_axis, mean_history - std_history, mean_history + std_history, alpha=0.2, color='red')
        else:
            plt.plot(x_axis, mean_history, label=key)
            plt.fill_between(x_axis, mean_history - std_history, mean_history + std_history, alpha=0.3)

    # Styblinski-Tang の 5次元最適値の理論値
    global_optimum = -39.16599 * 5  # 5次元の場合
    plt.axhline(y=global_optimum, color='r', linestyle='--', label="Global Optimum")

    plt.xlabel("Iteration")
    plt.ylabel("Best Value (Posterior Mean)")
    plt.title("Convergence Plot")
    plt.legend()
    plt.show()

  0%|          | 0/50 [elapsed: 00:00 remaining: ?]